# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.235443893334                   -0.50    8.0         
  2   -7.250309583765       -1.83       -1.40    1.0   6.99ms
  3   -7.251091973630       -3.11       -2.03    1.0   6.96ms
  4   -7.251052803152   +   -4.41       -1.93    2.0   8.58ms
  5   -7.251333826772       -3.55       -2.71    1.0   6.83ms
  6   -7.251338198171       -5.36       -3.18    1.0   6.83ms
  7   -7.251338689588       -6.31       -3.50    2.0   8.51ms
  8   -7.251338788722       -7.00       -4.04    1.0   6.98ms
  9   -7.251338796288       -8.12       -4.30    3.0   9.95ms
 10   -7.251338798183       -8.72       -4.72    1.0   7.34ms
 11   -7.251338798662       -9.32       -5.30    2.0   8.80ms
 12   -7.251338798696      -10.48       -5.50    3.0   10.3ms
 13   -7.251338798704      -11.10       -6.19    1.0   7.19ms
 14   -7.251338798704      -12.08       -6.40    3.0   10.7ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.06843915734409153
[ Info: Arnoldi iteration step 2: normres = 0.4164532561112056
[ Info: Arnoldi iteration step 3: normres = 1.0098780613328449
[ Info: Arnoldi iteration step 4: normres = 0.2971934147623961
[ Info: Arnoldi iteration step 5: normres = 0.5922441918274343
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (5.79e-02, 7.77e-02, 5.25e-01, 2.55e-01, 2.22e-02)
[ Info: Arnoldi iteration step 6: normres = 0.2322958096482033
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.05e-02, 8.37e-02, 1.66e-01, 1.08e-01, 3.96e-02)
[ Info: Arnoldi iteration step 7: normres = 0.0816171362237299
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (4.09e-04, 6.60e-03, 1.18e-02, 4.54e-02, 6.05e-02)
[ Info: Arnoldi iteration step 8: normres = 0.08586696873890254
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.49e-05